#Question 2: CIFAR-10 dataset reconstruction
Dataset Problem: Use the CIFAR-10 dataset to train an autoencoder. The goal is to input an image of a CIFAR-10 dataset into the autoencoder and have it reconstructed the image as a new output. Use the concept of VAE to solve this problem
CIFAR is an acronym that stands for the Canadian Institute for Advanced Research and the
CIFAR-10 dataset was developed along with the CIFAR-100 dataset by researchers at the CIFAR institute. The dataset is comprised of 60,000 32 X 32-pixel color photographs of objects from10 classes, such as frogs, birds, cats, ships, etc. The class labels and their standard associated
integer values are listed below.

0: airplane
1: automobile
2: bird
3: cat
4: deer
5: dog
6: frog
7: horse
8: ship
9: truck


In [1]:
import pickle

# Function to unpickle CIFAR-10 batch files
def unpickle(file):
    with open(file, 'rb') as fo:
        dict_data = pickle.load(fo, encoding='bytes')
    return dict_data

# List of uploaded files
file_paths = [
    "/content/batches.meta",
    "/content/data_batch_1",
    "/content/data_batch_2",
    "/content/data_batch_3",
    "/content/data_batch_4",
    "/content/data_batch_5",
    "/content/test_batch"
]

# Load and display the first batch as an example
data_batch_1 = unpickle(file_paths[1])

# Print keys in the dictionary to explore the data structure
print(data_batch_1.keys())

# Extract images and labels
images = data_batch_1[b'data']
labels = data_batch_1[b'labels']

# Display shape of the data
print(f"Shape of images array: {images.shape}")
print(f"Number of labels: {len(labels)}")

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])
Shape of images array: (10000, 3072)
Number of labels: 10000


#1. Data Preparation:

In [2]:
import tensorflow as tf

# Load CIFAR-10 dataset
(x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


#2. Define the VAE Architecture:

In [3]:
from tensorflow.keras import layers

latent_dim = 128  # Dimension of the latent space

# Encoder
encoder_inputs = tf.keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(32, 3, activation='relu', strides=2, padding='same')(encoder_inputs)
x = layers.Conv2D(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)

# Sampling function
def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = layers.Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# Decoder
decoder_inputs = tf.keras.Input(shape=(latent_dim,))
x = layers.Dense(8 * 8 * 64, activation='relu')(decoder_inputs)
x = layers.Reshape((8, 8, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same')(x)
x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same')(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation='sigmoid', padding='same')(x)

# Define encoder and decoder models
encoder = tf.keras.Model(encoder_inputs, [z_mean, z_log_var, z], name='encoder')
decoder = tf.keras.Model(decoder_inputs, decoder_outputs, name='decoder')

#3. Define the VAE Model with Custom Loss:

In [4]:
class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

# Instantiate the VAE model
vae = VAE(encoder, decoder)


#4. Compile and Train the Model:

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    def compute_loss(self, x, *args, **kwargs): # Modified line
        z_mean, z_log_var, z = self.encoder(x)
        x_reconstructed = self.decoder(z)

        # Reconstruction loss
        reconstruction_loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(x, x_reconstructed))

        # KL divergence loss
        kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)

        return reconstruction_loss + kl_loss

    def train_step(self, data):
        with tf.GradientTape() as tape:
            loss = self.compute_loss(data) # data contains x, y, sample_weight
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return {"loss": loss}

# Initialize model
vae = VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())

# Train the model
vae.fit(x_train, epochs=4, batch_size=127, validation_data=(x_test, None))

#5. Evaluate and Visualize the Results

In [ ]:
import matplotlib.pyplot as plt

# Select a few test images
num_images = 10
sample_images = x_test[:num_images]
reconstructed_images = vae.predict(sample_images)

# Plot original and reconstructed images
plt.figure(figsize=(20, 4))
for i in range(num_images):
    # Original images
    ax = plt.subplot(2, num_images, i + 1)
    plt.imshow(sample_images[i])
    plt.title("Original")
    plt.axis("off")

    # Reconstructed images
    ax = plt.subplot(2, num_images, i + 1 + num_images)
    plt.imshow(reconstructed_images[i])
    plt.title("Reconstructed")
    plt.axis("off")
plt.show()


#6.Code to Evaluate VAE Performance Using SSIM and MSE

In [ ]:
import numpy as np
import tensorflow as tf
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt

# Function to compute SSIM and MSE
def evaluate_reconstruction(vae, x_test):
    num_samples = 100  # Number of test images to evaluate
    x_test_sample = x_test[:num_samples]

    # Generate reconstructed images
    reconstructed_images = vae.predict(x_test_sample)

    ssim_scores = []
    mse_scores = []

    for i in range(num_samples):
        # Compute SSIM (Structural Similarity Index)
        ssim_score = ssim(x_test_sample[i], reconstructed_images[i], multichannel=True)
        ssim_scores.append(ssim_score)

        # Compute MSE (Mean Squared Error)
        mse_score = np.mean((x_test_sample[i] - reconstructed_images[i]) ** 2)
        mse_scores.append(mse_score)

    avg_ssim = np.mean(ssim_scores)
    avg_mse = np.mean(mse_scores)

    print(f"Average SSIM Score: {avg_ssim:.4f}")  # Higher is better (1 is perfect reconstruction)
    print(f"Average MSE Score: {avg_mse:.4f}")  # Lower is better (0 means identical images)

# Call the evaluation function
evaluate_reconstruction(vae, x_test)